In [1]:
import pandas as pd

# Đọc 3 file 
df_basics = pd.read_csv('title.basics.tsv', sep='\t', low_memory=False)
df_ratings = pd.read_csv('title.ratings.tsv', sep='\t', low_memory=False)
df_crew = pd.read_csv('title.crew.tsv', sep='\t', low_memory=False)

In [2]:
# THÊM THUỘC TÍNH CẦN THIẾT

In [3]:
# Tạo thêm thuộc tính primaryTitleLength bằng độ dài tên chính primaryTitle trong file title.basics.tsv --> thêm vào file title.basics
# Tạo thêm thuộc tính numGenres là số lượng thể loại được tính toán từ thuộc tính Genres trong file title.basics.tsv --> thêm vào file title.basics
# Tạo thêm thuộc tính writerCount là số lượng nhà văn được tính từ thuộc tính Writers trong file title.crew.tsv --> thêm vào file title.crew

In [4]:
# Đảm bảo tất cả các giá trị trong cột 'primaryTitle' là chuỗi và xử lý giá trị thiếu
df_basics['primaryTitle'] = df_basics['primaryTitle'].astype(str).fillna('')

# Tạo cột mới 'titleLength' chứa độ dài của mỗi giá trị trong cột 'primaryTitle'
df_basics['titleLength'] = df_basics['primaryTitle'].apply(lambda x: len(x))

# Tạo cột mới 'numGenres' là số lượng thể loại từ 'genres'
df_basics['numGenres'] = df_basics['genres'].apply(lambda x: len(x.split(',')) if pd.notnull(x) else 0)

# Tạo thêm cột 'writerCount' là số lượng nhà văn từ cột 'writers'
df_crew['writerCount'] = df_crew['writers'].apply(lambda x: len(x.split(',')) if pd.notnull(x) else 0)

In [5]:
# GỘP 3 FILE

In [6]:
# Gộp 3 files thông qua thuộc tính 'tconst'

# Tạo ra file mới 'merged_df'
merged_df = df_basics.merge(df_ratings, on='tconst', how='inner')  # Inner join (keeps all rows)
merged_df = merged_df.merge(df_crew, on='tconst', how='inner')  # Merge with crew data

# Chuyển file thành định dạng csv
merged_df.to_csv('merged_data.tsv', sep='\t', index=False)  # Without index column

# In ra thông số
print(merged_df.shape)

(1426606, 16)


In [7]:
# XỬ LÝ GIÁ TRỊ NULL

In [8]:
# Đếm số lượng dòng có ít nhất 1 giá trị null
missing_row_count = merged_df.isnull().any(axis=1).sum()
print(missing_row_count)

3


In [9]:
# Xử lý giá trị NULL
# gán merged_df vào 1 dataframe khác và sử dụng hàm dropna
filtered_df = merged_df.dropna()

# gán lại dataframe vào merged_df ban đầu 
merged_df = filtered_df

# in ra số thông số merged_df
print(merged_df.shape)

(1426603, 16)


In [10]:
# Xử lý cột 'genres'
# Tạo ra dataframe mới là filtered_df = merged_df

# Xoá các hàng có cột genres chứa giá trị '\N' 
filtered_df = merged_df[~merged_df['genres'].str.contains("\\\\N")]  

# in ra thông số filtered_df - thông số của dataframe sau khi xoá các phần tử NULL ở cột 'genres'
print("filtered_df: ")
print(filtered_df.shape)


filtered_df: 
(1406637, 16)


In [11]:
# Xử lý giá trị NULL với cột 'genres'
# gán filtered_df lại cho merged_df
merged_df = filtered_df

# In ra thông số của merged_df
print("merged_df: ")
print(merged_df.shape)

merged_df: 
(1406637, 16)


In [12]:
# Xử lý cột 'runtimeMinutes'
# Xoá các hàng có cột runtimeMinutes chứa giá trị '\N' 
filtered_df = merged_df[~merged_df['runtimeMinutes'].str.contains("\\\\N")]  

# in ra thông số filtered_df - thông số của dataframe sau khi xoá các phần tử NULL ở cột 'runtimeMinutes'
print("filtered_df: ")
print(filtered_df.shape)

filtered_df: 
(1003852, 16)


In [13]:
# Xử lý giá trị NULL với cột 'runtimeMinutes'

# gán filtered_df lại cho merged_df
merged_df = filtered_df

# In ra thông số của merged_df
print("merged_df: ")
print(merged_df.shape)

merged_df: 
(1003852, 16)


In [14]:
# XỬ LÝ PHẦN TỬ OUTLIERS

In [15]:
import pandas as pd
import numpy as np
from scipy import stats

# Hàm tìm outliers sử dụng z-scores
def identify_outliers_zscore(df, threshold=3):
    # chọn ra các cột có kiểu dữ liệu là số
  numerical_cols = df.select_dtypes(include=np.number) 
    #khởi tạo 1 dataframe rỗng
  outliers_df = pd.DataFrame() 
    # duyệt qua từng cột
  for col in numerical_cols:
      # tính absolute z-scores
    z_scores = np.abs(stats.zscore(df[col])) 
      # Lấy dòng với outlier trong từng cột
    outlier_rows = df[z_scores > threshold].index  
      # Thêm outlier vào dataframe
    outliers_df = pd.concat([outliers_df, df.loc[outlier_rows]], ignore_index=True)  
  return outliers_df.drop_duplicates()  
    # Xoá trùng (duplicate) nếu tồn tại


In [16]:
# In ra các phần tử bị outlier sau khi qua hàm
outlier_row_df = identify_outliers_zscore(merged_df.copy(), 3)
print(outlier_row_df)

          tconst  titleType  \
0      tt0000075      short   
1      tt0000166      short   
2      tt0000168      short   
3      tt0000349      short   
4      tt0000350      short   
...          ...        ...   
41289  tt9878574  tvEpisode   
41290  tt9878576  tvEpisode   
41291  tt9881616      movie   
41292  tt9897038   tvSeries   
41293  tt9900092   tvSeries   

                                            primaryTitle  \
0      The Conjuring of a Woman at the House of Rober...   
1      Salida de los trabajadores de la fábrica Españ...   
2      Salida del público de la iglesia parroquial de...   
3      What Came Out of the Cheese; or, The Lilliputi...   
4      The Countryman's First Sight of the Animated P...   
...                                                  ...   
41289                                       Karamo Brown   
41290                                        Jacob Tobia   
41291                                    La filla d'algú   
41292                      

In [17]:
# Tạo fitered_df = merged_df và xoá những hàng có chứa ít nhất 1 phần tử bị outlier
filtered_df = merged_df[~merged_df.index.isin(outlier_row_df.index)]  # Exclude rows in outlier_indices

# Gán filtered_df vào lại cho merged_df
merged_df = filtered_df

# In ra thông số của merged_df
print(merged_df.shape)


(965324, 16)


In [18]:
# XỬ LÝ PHẦN TỬ BỊ NHIỄU THEO BUSINESS LOGIC

In [19]:
# Loại bỏ dữ liệu nhiễu sử dụng business logic
# Thông thường, một bộ phim có trên 1000 lượt đánh giá sẽ được xem xét là đủ khách quan
# Lọc ra cá phần tử có lượt đánh giá nhỏ hơn 1000
filtered_df = merged_df[merged_df['numVotes'] < 1500]

# Print the filtered DataFrame
print(filtered_df.shape)


(905530, 16)


In [20]:
# Tiến hành lọc ra các phần tử có trên 1000 lượt đánh giá
filtered_df = merged_df[merged_df['numVotes'] > 1000]

# Gán filtered_df vào cho merged_df
merged_df = filtered_df.copy()

# In ra thông số của merged_df
print("merged_df: ")
print(merged_df.shape)

merged_df: 
(79335, 16)


In [21]:
# XỬ LÝ CÁC PHẦN TỬ TRÙNG NHAU

In [22]:
# Tìm các phần tử bị trùng nhau ở cột primaryTitle
dup_row = merged_df[merged_df.duplicated(subset='primaryTitle', keep=False)]
print(dup_row)

            tconst  titleType        primaryTitle       originalTitle isAdult  \
22837    tt0040446      movie          Homecoming          Homecoming       0   
23979    tt0041855      movie   The Secret Garden   The Secret Garden       0   
24255    tt0042210      movie              Utopia             Atoll K       0   
24399    tt0042367      movie  Cyrano de Bergerac  Cyrano de Bergerac       0   
24404    tt0042372      movie              Dallas              Dallas       0   
...            ...        ...                 ...                 ...     ...   
1426183  tt9906260  tvEpisode                Hero                Hero       0   
1426213  tt9907782      movie          The Cursed    Eight for Silver       0   
1426243  tt9908860   tvSeries          Blown Away          Blown Away       0   
1426261  tt9909484  tvEpisode       All About Eve       All About Eve       0   
1426575  tt9916362      movie               Coven            Akelarre       0   

        startYear endYear r

In [23]:
# Tìm các phần tử bị trùng nhau ở cột primaryTitle hoặc cột originalTitle
merged_df_duplicates = merged_df[merged_df.duplicated(subset=['primaryTitle', 'originalTitle'])]
print(merged_df_duplicates.shape)

(8558, 16)


In [24]:
# Xử lý các phần tử trùng nhau ở 2 cột

# Tạo merged_df_without_duplicates = merged_df sau khi xoá các phần tử và giữ lại phần tử đầu tiên
merged_df_without_duplicates = merged_df.drop_duplicates(subset=['primaryTitle', 'originalTitle'], keep = 'first')

# Gán merged_df_without_duplicates vào merged_df ban đầu
merged_df = merged_df_without_duplicates

# In ra thông số
print(merged_df.shape)

(70777, 16)


In [25]:
## file cleadned_data.csv đã có sẵn, nếu chưa thì hãy chạy cell code ở dưới

# Xuất dữ liệu DataFrame thành file CSV
file_path = 'cleaned_data.csv'
merged_df.to_csv(file_path, index=False)